In [7]:
from dotenv import load_dotenv
load_dotenv()

True

In [8]:
from langchain import hub
from langchain_core.messages import  HumanMessage
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.document_loaders import TextLoader
import os
from langchain.sql_database import SQLDatabase
from langchain.chains import create_extraction_chain
from langchain_community.agent_toolkits import create_sql_agent

In [9]:
loader = TextLoader("./data.txt")
docs = loader.load()
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# splits = text_splitter.split_documents(docs) #if we need to split doc.
vectorstore = Chroma.from_documents(documents=docs, embedding=OpenAIEmbeddings())

schema = {
    "properties": {
        "name": {"type": "string"},
        "phone_number": {"type": "integer"},
    },
    "required": [],
}

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [10]:
contextualize_q_system_prompt = """Imagine you are a receptionist at South Bay Dental Office. """
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)
contextualize_q_chain = contextualize_q_prompt | llm | StrOutputParser()

In [11]:
# DB_URI = "sqlite:///./Chinook.db"
# print(DB_URI)
# db = SQLDatabase.from_uri(DB_URI)

# create_data_llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
# create_agent_executor = create_sql_agent(create_data_llm, db=db, agent_type="openai-tools", verbose=False)
# create_agent_executor.invoke(
#     f"""Create Table name appointment table if already not exist. where colums are Phone Number : Integer, Name : String, Date : Date , Time:time fields."""
# )

In [12]:
qa_system_prompt = """Imagine you are a receptionist at South Bay Dental Office. \
    {context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

def contextualized_question(input: dict):
    if input.get("chat_history"):
        return contextualize_q_chain
    else:
        return input["question"]


rag_chain = (
    RunnablePassthrough.assign(
        context=contextualized_question | retriever
    )
    | qa_prompt
    | llm
)

In [13]:
chat_history = []

llmdata = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


def check_phone_name(converted_answer):
  if converted_answer.get('phone_number') and converted_answer.get('phone_number') != 1234567890:
    print("Adding Phone and name into database")
    # DB_URI = f"mysql://{os.environ['DB_USER']}:{os.environ['DB_PASSWORD']}@{os.environ['DB_HOST']}:{os.environ['DB_PORT']}/{os.environ['DB_NAME']}"
    DB_URI = "sqlite:///Chinook.db"
    print(DB_URI)
    db = SQLDatabase.from_uri(DB_URI)
  
    insert_data_llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
    agent_executor = create_sql_agent(insert_data_llm, db=db, agent_type="openai-tools", verbose=True)
    agent_executor.invoke(
      # f"Add phone_number : {converted_answer.get('phone_number')} and name : {converted_answer.get('name')} insert data into appointment table make date and time as current date and time."
      f"""Insert the following data into the appointment table:

      - Phone Number: {converted_answer.get('phone_number')}
      - Name: {converted_answer.get('name')}

      """
    )
    
for i in range(5):
  
  question = input("Himalay: ")
  print("Himlay: ", question)
  
  
  chain = create_extraction_chain(schema, llmdata)
  converted_answer = chain.run(question)
  if len(converted_answer) > 0:
    check_phone_name(converted_answer[0])
  print(f"user converted answer - {converted_answer}")
  
  ai_msg = rag_chain.invoke({"question": question, "chat_history": chat_history})
  
  
  print("AI Response: ",ai_msg)
  print("--------------------------------------------------------------")
  chat_history.extend([HumanMessage(content=question), ai_msg])

Himlay:  riya 9624788668


/Users/himalaypatel/Documents/HimalayWork/Learning/LangChain-AI-Voice-Call-asistant/venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Adding Phone and name into database
sqlite:///Chinook.db


> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with `{}`


appointment
Invoking: `sql_db_schema` with `{'table_names': 'appointment'}`



CREATE TABLE appointment (
	id INTEGER, 
	name TEXT DEFAULT '', 
	phone_number INTEGER DEFAULT '', 
	PRIMARY KEY (id)
)

/*
3 rows from appointment table:
id	name	phone_number
1	Himalay	9624788668
*/The appointment table has the following schema:

- id: INTEGER
- name: TEXT
- phone_number: INTEGER

To insert the data into the appointment table, I will use the following query:

```sql
INSERT INTO appointment (phone_number, name)
VALUES (9624788668, 'riya')
```

> Finished chain.
user converted answer - [{'name': 'riya', 'phone_number': 9624788668}]


Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2
